In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from Energy_Models.Autoformer import AutoFormer


In [2]:
df = pd.read_csv("C:\\Users\\delll\\Downloads\\energydata_complete.csv",index_col="date")

In [3]:
scaler = MinMaxScaler().fit(df)
df    = pd.DataFrame(scaler.transform(df))

X = df.iloc[:,1:]
y = df.iloc[:,0]

In [4]:
from sklearn.model_selection import train_test_split
X_train ,X_test ,y_train ,y_test=train_test_split(X ,y ,test_size=0.30,random_state=35)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.50, random_state=42)
X_train = np.array(X_train)
X_test  = np.array(X_test)

In [5]:
X_train.shape ,X_test.shape ,y_train.shape ,y_test.shape

((13814, 27), (2961, 27), (13814,), (2961,))

In [6]:
X_val = np.array(X_val)

In [7]:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test  = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

X_val = X_val.reshape(X_val.shape[0],1,X_val.shape[1])

In [8]:
X_train.shape ,X_test.shape 

((13814, 1, 27), (2961, 1, 27))

In [9]:
def root_mean_squared_error(y_true, y_pred):    
    return np.sqrt(mean_squared_error(y_true, y_pred))
model=AutoFormer(1,27,1).getModel()

In [10]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 27)]      0           []                               
                                                                                                  
 encoder_layer_1 (EncoderLayer)  (None, 1, 27)       756         ['input_1[0][0]']                
                                                                                                  
 tf.__operators__.getitem_3 (Sl  (None, 27)          0           ['encoder_layer_1[0][0]']        
 icingOpLambda)                                                                                   
                                                                                                  
 dropout_4 (Dropout)            (None, 27)           0           ['tf.__operators__.getitem_3[

In [ ]:
model.compile(optimizer='adam', loss ='mse')
history = model.fit(X_train , y_train, epochs=100, batch_size=64, verbose= 1,validation_split=0.2,callbacks=callback)

Epoch 1/100
173/173 [==============================] - 13s 61ms/step - loss: 0.1670 - val_loss: 0.1505
Epoch 2/100
173/173 [==============================] - 9s 53ms/step - loss: 0.1363 - val_loss: 0.1226
Epoch 3/100
173/173 [==============================] - 10s 56ms/step - loss: 0.1110 - val_loss: 0.0998
Epoch 4/100
173/173 [==============================] - 9s 51ms/step - loss: 0.0906 - val_loss: 0.0816
Epoch 5/100
173/173 [==============================] - 9s 53ms/step - loss: 0.0743 - val_loss: 0.0671
Epoch 6/100
173/173 [==============================] - 11s 61ms/step - loss: 0.0613 - val_loss: 0.0556
Epoch 7/100
173/173 [==============================] - 9s 50ms/step - loss: 0.0511 - val_loss: 0.0465
Epoch 8/100
173/173 [==============================] - 9s 51ms/step - loss: 0.0430 - val_loss: 0.0393
Epoch 9/100
173/173 [==============================] - 11s 65ms/step - loss: 0.0365 - val_loss: 0.0335
Epoch 10/100
173/173 [==============================] - 9s 53ms/step - loss: 0

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error, mean_absolute_percentage_error
def root_mean_squared_error(y_true, y_pred):    
    return np.sqrt(mean_squared_error(y_true, y_pred))
def root_mean_squared_log_error(real, predicted):
    sum=0.0
    for x in range(len(predicted)):
        if predicted[x]<0 or real[x]<0: # check for negative values
            continue
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5
def print_metrics():
    print('Train   RMSE  value   : %.3f ' % root_mean_squared_error(y_train, model.predict(X_train)))
    print('Train   MSE   value   : %.3f ' % mean_squared_error(y_train, model.predict(X_train)))
    print('Train   R2    value   : %.3f ' % r2_score(y_train,model.predict(X_train)))
    print('Train   MAPE  value   : %.3f ' % mean_absolute_percentage_error(y_train, model.predict(X_train)))
    print('Train   MAE   value   : %.3f ' % mean_absolute_error(y_train, model.predict(X_train)))
    print('---------------------------------------------')
    print('Test    RMSE  value   : %.3f ' % root_mean_squared_error(y_test, model.predict(X_test)))
    print('Test    MSE   value   : %.3f ' % mean_squared_error(y_test, model.predict(X_test)))
    print('Test    R2    value   : %.3f ' % r2_score(y_test, model.predict(X_test)))
    print('Test    MAPE  value   : %.3f ' % mean_absolute_percentage_error(y_test, model.predict(X_test)))
    print('Test    MAE   value   : %.3f ' % mean_absolute_error(y_test, model.predict(X_test)))

In [ ]:
print_metrics()

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Plot of loss vs epoch for train and test dataset")
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()